# Install and Import dependencies

In [1]:
!pip install torch torchvision torchaudio

                                              0.0/172.3 MB ? eta -:--:--
                                              0.1/172.3 MB 3.6 MB/s eta 0:00:48
                                              0.7/172.3 MB 8.7 MB/s eta 0:00:20
                                             1.5/172.3 MB 11.9 MB/s eta 0:00:15
                                             2.5/172.3 MB 14.4 MB/s eta 0:00:12
                                             3.4/172.3 MB 15.5 MB/s eta 0:00:11
     -                                       4.9/172.3 MB 17.4 MB/s eta 0:00:10
     -                                       6.0/172.3 MB 18.3 MB/s eta 0:00:10
     -                                       7.4/172.3 MB 19.0 MB/s eta 0:00:09
     -                                       8.4/172.3 MB 19.2 MB/s eta 0:00:09
     --                                     10.1/172.3 MB 20.2 MB/s eta 0:00:09
     --                                     11.6/172.3 MB 23.4 MB/s eta 0:00:07
     --                                     13.

In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [3]:
%pip install transformers


In [96]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

# Instantiate Model

In [9]:
# https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Test on spanish data

In [41]:
# Specify the file path relative to the current Python file
file_path = '../Preprocessing/data_clean/es_clean_1.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

,data,player,language,publishedAt
224,Bayer Leverkusen y Mónaco jugaron en la Europa...,Piero Hincapié,es,2023-02-16T22:09:36Z
225,La Federación Ecuatoriana de Fútbol (FEF) invi...,Piero Hincapié,es,2023-02-14T23:40:33Z
226,Los abogados de la Federación Ecuatoriana de F...,Piero Hincapié,es,2023-02-14T15:58:38Z
227,Usamos cookies a fin de mejorar nuestro servic...,Piero Hincapié,es,2023-02-03T10:58:00Z
228,«Queremos anunciar la incorporación a IDV de n...,Piero Hincapié,es,2023-02-02T21:53:12Z


In [67]:
exequiel_palacios_df = df[df['player'] == 'Exequiel Palacios']

exequiel_palacios_df['data'].iloc[0]

"Mercado de fichajes de fútbol, en directoAdeyemi firmó el que es su primer gol en lo que va de BundesligaJavier AlfaroRedactor FútbolEl que perdona, lo acaba pagando muy caro en esto del fútbol. En España, en la China y en Alemania. En la liga de este último país lo sufrió este domingo el Bayer Leverkusen de Xabi Alonso, que fue indulgente con el Borussia Dortmund y contra su voluntad acabó cosechando una derrota de esas que se califican como injustas por los méritos realizados. Pero la realidad es que en esto del balompié, justicia a parte, gana el que marca más goles que el rival, y mientras que el Dortmund hizo dos el Leverkusen no logró ninguno (0-2).Con Florian Wirtz dejando su lesión definitivamente en el olvido y Exequiel Palacios y Amiri creando juego en la medular, el Bayer Leverkusen llevó la iniciativa en el juego desde el inicio, haciendo latente lo bien trabajando que tiene el equipo el bueno de Xabi Alonso, pero ni los mencionados ni sus compañeros de equipo fueron capac

In [64]:
def sentiment_score(data):
    tokens = tokenizer.encode(data, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [68]:
exequiel_palacios_df['sentiment'] = exequiel_palacios_df['data'].apply(lambda x: sentiment_score(x[:512]))
exequiel_palacios_df

/var/folders/fq/0lb0fh614nz4_s73hrgyrcd00000gn/T/ipykernel_38677/3539805919.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exequiel_palacios_df['sentiment'] = exequiel_palacios_df['data'].apply(lambda x: sentiment_score(x[:512]))


,data,player,language,publishedAt,sentiment
0,"Mercado de fichajes de fútbol, en directoAdeye...",Exequiel Palacios,es,2023-01-29T18:25:03Z,1
1,La última semana de marzo es la primera ventan...,Exequiel Palacios,es,2023-01-30T16:52:46Z,2
2,Gustavo Puerta ya no jugará en el Bayer Leverk...,Exequiel Palacios,es,2023-01-31T20:41:38Z,1
3,Alberto FernándezMauricio MacriINDECHarvardBoc...,Exequiel Palacios,es,2023-02-09T18:32:38Z,1
4,Alberto FernándezMauricio MacriINDECHarvardBoc...,Exequiel Palacios,es,2023-02-12T21:13:55Z,2
...,...,...,...,...,...
79,Usamos cookies a fin de mejorar nuestro servic...,Exequiel Palacios,es,2023-04-13T17:04:00Z,4
80,El máximo ganador de la historia de la compete...,Exequiel Palacios,es,2023-04-20T21:55:59Z,5
81,Usamos cookies a fin de mejorar nuestro servic...,Exequiel Palacios,es,2023-04-28T10:28:00Z,3
82,La Liga Profesional tendrá como principal ofer...,Exequiel Palacios,es,2023-04-29T03:41:10Z,3


In [69]:
# Calculate the mean of the column
column_mean = exequiel_palacios_df['sentiment'].mean()

# Print the mean value
print(column_mean)

3.0238095238095237


In [80]:
exequiel_palacios_df['data'].iloc[80]

'El máximo ganador de la historia de la competencia fue demasiado para un elenco inglés dormido que no logró hacer pie en el estadio Ramón Sánchez-Pizjuán en ningún momento. El resultado global terminó 5 a 2 luego de un 2 a 2 en Inglaterra (goles de Marcel Sabitzer para los Diablos Rojos y de Harry Maguire y Tyrell Malacia en contra para los Nervionenses) y un contundente 3 a 0 para el equipo local en España (dos tantos de Youssef En-Nesyri y uno de Loic Badé). Así, el conjunto dirigido por José Luis Mendilibar aspirará a meterse en su séptima final de Europa League con una particularidad: ganó las seis que disputó.Por otro lado, a la Vecchia Signora le alcanzó un empate 1 a 1 en el Allianz Stadium (anotaciones de Adrien Rabiot para el elenco italiano y de Marcus Edwards para el combinado portugués) para sentenciar la serie 2 a 1 ante el conjunto portugués luego del 1 a 0 como visitante en el encuentro de ida (gol de Frederico Gatti). De esta manera, los dirigidos por Massimiliano Alle

# single tests

In [76]:
tokens = tokenizer.encode('El club alemán, que siempre se ha caracterizado por confiar en gente joven con sangre nueva y, por supuesto, en mediocampistas sudamericanos con sacrificio, físico y bien dotados técnicamente, como lo son el chileno Charles Aranguiz que actualmente se encuentra lesionado y su regreso se espera para mitad de febrero, y el Argentino de 24 años Exequiel Palacios.', return_tensors='pt')
result = model(tokens)
result.logits
int(torch.argmax(result.logits))+1

3

In [97]:
def extract_sentence(df):
    df['short'] = ''
    
    for index, row in df.iterrows():
        player = row['player']
        data = str(row['data'])  # Convert data to string
        
        # Find the sentence containing player's information
        sentences = data.split('.')
        for sentence in sentences:
            if player.lower() in sentence.lower():
                df.at[index, 'short'] = sentence.strip()
                break
    
    return df

In [102]:
extract_sentence(exequiel_palacios_df)


/var/folders/fq/0lb0fh614nz4_s73hrgyrcd00000gn/T/ipykernel_38677/1634877259.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['short'] = ''


,data,player,language,publishedAt,sentiment,short,sentiment2
0,En la liga de este último país lo sufrió este...,Exequiel Palacios,es,2023-01-29T18:25:03Z,1,Con Florian Wirtz dejando su lesión definitiva...,2
1,Los oponentes se confirmarán luego de que Sca...,Exequiel Palacios,es,2023-01-30T16:52:46Z,2,"Además de Messi y Ángel Di María, máximos refe...",4
2,Bayer Leverkusen pescó a una de las figuras ma...,Exequiel Palacios,es,2023-01-31T20:41:38Z,1,"El club alemán, que siempre se ha caracterizad...",3
3,"En este caso, la satisfacción tiene que ver co...",Exequiel Palacios,es,2023-02-09T18:32:38Z,1,"El presidente de la AFA, Claudio \'Chiqui\' Ta...",4
4,Se lo podría observar y analizar como a un se...,Exequiel Palacios,es,2023-02-12T21:13:55Z,2,También participaron los otros campeones de la...,4
...,...,...,...,...,...,...,...
79,En su primera temporada como entrenador de un ...,Exequiel Palacios,es,2023-04-13T17:04:00Z,4,Pero ensalzar las recientes victorias consecut...,4
80,NaN,Exequiel Palacios,es,2023-04-20T21:55:59Z,5,,4
81,El mediocampista argentino Exequiel Palacios v...,Exequiel Palacios,es,2023-04-28T10:28:00Z,3,El mediocampista argentino Exequiel Palacios v...,2
82,"Y más tarde lo harán Boca y Racing, equipos q...",Exequiel Palacios,es,2023-04-29T03:41:10Z,3,Alexis Mac Allister intentará acercar a Bright...,3


In [99]:
exequiel_palacios_df['sentiment2'] = exequiel_palacios_df['short'].apply(lambda x: sentiment_score(x[:512]))
exequiel_palacios_df

/var/folders/fq/0lb0fh614nz4_s73hrgyrcd00000gn/T/ipykernel_38677/3692395472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exequiel_palacios_df['sentiment2'] = exequiel_palacios_df['short'].apply(lambda x: sentiment_score(x[:512]))


,data,player,language,publishedAt,sentiment,short,sentiment2
0,En la liga de este último país lo sufrió este...,Exequiel Palacios,es,2023-01-29T18:25:03Z,1,Con Florian Wirtz dejando su lesión definitiva...,2
1,Los oponentes se confirmarán luego de que Sca...,Exequiel Palacios,es,2023-01-30T16:52:46Z,2,"Además de Messi y Ángel Di María, máximos refe...",4
2,Bayer Leverkusen pescó a una de las figuras ma...,Exequiel Palacios,es,2023-01-31T20:41:38Z,1,"El club alemán, que siempre se ha caracterizad...",3
3,"En este caso, la satisfacción tiene que ver co...",Exequiel Palacios,es,2023-02-09T18:32:38Z,1,"El presidente de la AFA, Claudio \'Chiqui\' Ta...",4
4,Se lo podría observar y analizar como a un se...,Exequiel Palacios,es,2023-02-12T21:13:55Z,2,También participaron los otros campeones de la...,4
...,...,...,...,...,...,...,...
79,En su primera temporada como entrenador de un ...,Exequiel Palacios,es,2023-04-13T17:04:00Z,4,Pero ensalzar las recientes victorias consecut...,4
80,NaN,Exequiel Palacios,es,2023-04-20T21:55:59Z,5,,4
81,El mediocampista argentino Exequiel Palacios v...,Exequiel Palacios,es,2023-04-28T10:28:00Z,3,El mediocampista argentino Exequiel Palacios v...,2
82,"Y más tarde lo harán Boca y Racing, equipos q...",Exequiel Palacios,es,2023-04-29T03:41:10Z,3,Alexis Mac Allister intentará acercar a Bright...,3


In [101]:
# Calculate the mean of the column
column_mean = exequiel_palacios_df['sentiment2'].mean()

# Print the mean value
print(column_mean)

3.3333333333333335


In [196]:
exequiel_palacios_df['short'].iloc[66]

'Bayern viene de resignar el primer puesto de la Bundesliga tras perder ante Bayer Leverkusen (dos goles de penal de Exequiel Palacios) y tendrá la posibilidad de recuperarlo el próximo sábado, cuando recibirá al nuevo líder, Borussia Dortmund'

In [174]:
tokens = tokenizer.encode(exequiel_palacios_df['short'].iloc[43], return_tensors='pt')
result = model(tokens)
result.logits
int(torch.argmax(result.logits))+1

3